In [ ]:
# default_exp utils.functional

In [ ]:
# hide
from ipynb_path import *

In [ ]:
# export
from counternet.import_essentials import *
import functools


Global seed set to 31


In [ ]:
# export
def _check_type(X):
    if not torch.is_tensor(X):
        if isinstance(X, np.ndarray):
            X = torch.from_numpy(X)
        elif isinstance(X, list):
            X = torch.tensor(X)
        elif isinstance(X, pd.DataFrame):
            X = X.to_numpy()
            X = torch.from_numpy(X)
        elif isinstance(X, pd.Series):
            X = X.values
            X = torch.tensor(X)
        else:
            raise ValueError(f'input X should be one of these types: [`list`, `pd.DataFrame`, `np.ndarray`, `torch.Tensor`], but got {type(X)}')
    return X.float()

In [ ]:
_ =_check_type([1, 2])
_ =_check_type(np.array([1, 2]))
_ =_check_type(pd.read_csv('assets/data/dummy_data.csv'))
_ =_check_type(pd.read_csv('assets/data/adult.csv')['income'])

32561

In [ ]:
# export
def check_input_type(func):
    """check if all inputs are torch.Tensor"""
    @functools.wraps(func) 
    def wrapper_check_input_type(*args):
        new_args = []
        for X in list(args):
            new_args.append(_check_type(X))
        return func(*new_args)
    return wrapper_check_input_type

In [ ]:
@check_input_type
def plus_two(x):
    return x + 2

_x = [1, 2]
_x_np = np.array(_x)
_x_torch = torch.tensor(_x)
assert isinstance(plus_two(_x), torch.Tensor)
assert isinstance(plus_two(_x_np), torch.Tensor)
assert isinstance(plus_two(_x_torch), torch.FloatTensor)


In [ ]:
@check_input_type
def plus_two(x, y):
    return x + y + 2

_y = [1, 2]
_y_np = np.array(_y)
_y_torch = torch.tensor(_y)
assert isinstance(plus_two(_x, _y), torch.Tensor)
assert isinstance(plus_two(_x_np, _y_np), torch.Tensor)
assert isinstance(plus_two(_x_torch, _y_torch), torch.FloatTensor)


In [ ]:
# export
def check_object_input_type(func):
    """check if all inputs are torch.Tensor"""
    @functools.wraps(func) 
    def wrapper_check_input_type(ref, *args):
        new_args = [ref]
        for X in list(args):
            new_args.append(_check_type(X))
        return func(*new_args)
    return wrapper_check_input_type

In [ ]:
%%timeit -n 1000
class PlusTwo(object):
    @check_object_input_type
    def compute(self, x, y):
        return x + y + 2

test_obj = PlusTwo()

assert isinstance(test_obj.compute(_x, _y), torch.Tensor)
assert isinstance(test_obj.compute(_x_np, _y_np), torch.Tensor)
assert isinstance(test_obj.compute(_x_torch, _y_torch), torch.FloatTensor)

assert torch.equal(test_obj.compute(_x, _y), torch.tensor([4., 6.]))


122 µs ± 13.8 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
